In [3]:
query = input('검색어를 입력해주세요. : ')
start_date = input('검색 시작 날짜를 입력해주세요. (형식 : 2022.01.01) : ')
end_date = input('검색 종료 날짜를 입력해주세요. (형식 : 2022.12.31) : ')
sort_type = input('정렬 타입을 입력해주세요. (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : ')
max_page = input('크롤링을 원하는 페이지 수를 입력해주세요. : ')

if start_date > end_date :
    print('\n시작 날짜는 종료 날짜보다 이후로 지정 불가합니다.')
elif max_page == '':
    max_page = 5
    print('\n원하는 페이지 수가 입력되지 않았습니다. 5페이지 까지 크롤링 진행')
   # main_crawling(query, start_date, end_date, sort_type, max_page)
else:
    max_page = int(max_page)
    #main_crawling(query, start_date, end_date, sort_type, max_page)

검색어를 입력해주세요. : 데이터
검색 시작 날짜를 입력해주세요. (형식 : 2022.01.01) : 
검색 종료 날짜를 입력해주세요. (형식 : 2022.12.31) : 
정렬 타입을 입력해주세요. (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : 0
크롤링을 원하는 페이지 수를 입력해주세요. : 10


In [4]:
def main_crawling(query, start_date, end_date, sort_type, max_page):
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2022.01.01'
    if len(end_date) != 10:
        end_date = '2022.12.31'
    if sort_type not in ['0','1','2']:
        sort_type = '0'
    
    # 각 기사들을 담을 리스트 생성
    titles = []
    dates = []
    articles = []
    article_urls = []
    press_companies = []
    
    # 주어진 일자를 쿼리에 맞는 형태로 변경
    start_date = start_date.replace(".","")
    end_date = end_date.replace(".","")    
    
    # 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링
    current_call = 1
    last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당
    # 1 11 21 31 41....
    
    while current_call <= last_call:
        print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
        
        